In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_cardio = pd.read_csv('./data/cardio_train.csv', sep = ';')
df_cardio

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


In [2]:
df_cardio['ap_lo'].min(), df_cardio['ap_hi'].min(), df_cardio['ap_lo'].max(), df_cardio['ap_hi'].max()

(-70, -150, 11000, 16020)

## Metrics overview
- ap_hi is Systolic blood pressure. Normal 120 or little below, max 200, considered low under 90. Hard to find realistic lowest viable value, maybe 60 ? In some extreme case some humans can survive up to 300.
- ap_lo Diastolic blood pressure Normal 80,  Max 140, considered low under 60 unrealistic under 40 ?
- height 251 max

In [3]:
df_cardio['age_years'] = df_cardio['age'].apply(lambda x : x/365)
df_cardio['age_years'].min(), df_cardio['age_years'].max()
#Age range seems all good

(29.583561643835615, 64.96712328767123)

In [8]:
data_filters = ['ap_hi > 300 or ap_hi < 60 ', 'height < 80', 'weight < 30']
for condition in data_filters:
    df_cardio.drop(df_cardio.query(condition).index, axis=0, inplace = True)

df_cardio.info()

#df_cardio_hi = df_cardio.query('ap_hi > 300')
#df_cardio_lo = df_cardio.query('ap_hi < 60')
#df_cardio_max_height = df_cardio.query('height < 80')
#df_cardio_max_height
#No very small persons, associated weight indicates data error.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69744 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           69744 non-null  int64  
 1   age          69744 non-null  int64  
 2   gender       69744 non-null  int64  
 3   height       69744 non-null  int64  
 4   weight       69744 non-null  float64
 5   ap_hi        69744 non-null  int64  
 6   ap_lo        69744 non-null  int64  
 7   cholesterol  69744 non-null  int64  
 8   gluc         69744 non-null  int64  
 9   smoke        69744 non-null  int64  
 10  alco         69744 non-null  int64  
 11  active       69744 non-null  int64  
 12  cardio       69744 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 7.4 MB


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio


In [24]:
df_cardio.query('weight < 30')

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
26806,38312,23284,1,157,23.0,110,80,1,1,0,0,1,0
29488,42156,20408,2,177,22.0,120,80,1,1,1,1,1,0
33817,48318,21582,2,178,11.0,130,90,1,1,0,0,1,1
34276,48976,14664,2,128,28.0,120,80,1,1,0,0,1,0
57858,82567,18804,2,165,10.0,180,1100,2,2,0,0,1,1
60188,85931,21855,1,162,21.0,120,80,2,1,0,0,1,1
60699,86650,18875,1,171,29.0,110,70,2,1,0,0,1,1
